In [1]:
import numpy as np
import gym
import time

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from gym import Env, spaces
from gym.utils import seeding
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


In [10]:
class TestEnv(Env):
    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self) -> None:
        self.__max_step = 9
        
        # 0-8分别代表此次落子位置
        self.action_space = spaces.Discrete(9)
        # 初始全零，下标0为：1、2分别代表黑白两方（1黑方先行），下标1-9为：0空、1黑方、2白方
        self.observation_space = spaces.Box(
            np.zeros(10, int), np.zeros(10, int) + 2)
        # 代表AI所属方（1或2）
        self.ai = None
        self._seed()
        self.state = None
        self.__step = None

    def _seed(self, seed=None):
        # 随机AI所属方
        self.ai = np.random.choice([1, 2])
    
    def _reset(self):
        self.__step = 0
        self.state = [self.ai]
        self.state = self.state.extend(np.zeros(9, int).tolist())
        return np.array(self.state)

    def _step(self, action):
        grid = self.state[1:]
        # 不可重复落子
        if grid[action] != 0:
            return np.array(self.state), -100, True, {}
        # 黑方先手
        grid = np.array(grid, int)
        dif = np.sum(grid == 1) - np.sum(grid == 2)
        if dif == 0:
            current = 1
        elif dif == 1:
            current = 2
        else:
            return np.array(self.state), -100, True, {}
        # 胜负
        grid[action] = current
        self.state = [self.ai]
        self.state = self.state.extend(grid.tolist())
        is_win = lambda b: b[0:3].all() or b[3:6].all() or b[6:9].all() or \
            b[0::3].all() or b[1::3].all() or b[2::3] or \
            b[0::4].all() or b[2:7:2].all()
        if is_win(grid == current):
            if self.ai == current:
                reward = 10
            else:
                reward = -10
            return np.array(self.state), reward, True, {}

        reward = 0
        self.__step += 1
        if self.__step < self.__max_step:
            done = False
        else:
            done = True
        return np.array(self.state), reward, done, {}

    def _render(self, mode='ansi', close=False):
        print(self.state[0], np.array(self.state[1:], int).reshape(3, 3))

    def _close(self):
        super()._close()

In [11]:
env = TestEnv()
env.reset()
nb_actions = env.action_space.n

In [12]:
for i in range(10):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    print(env.render())
    print(action, reward, done, info)

TypeError: 'NoneType' object is not subscriptable

In [45]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                48        
_________________________________________________________________
activation_4 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_5 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 34        
_________________________________________________________________
activation_6 (Activation)    (None, 2)                 0         
Total para

In [46]:
memory = SequentialMemory(limit=500, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [47]:
dqn.fit(env, nb_steps=500, visualize=False, verbose=2)

Training for 500 steps ...
[0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 1]
[0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 1, 0, 1, 0, 1]
[0, 0, 0, 1, 0, 1, 0, 1, 0]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
  10/500: episode: 1, duration: 0.125s, episode steps: 10, steps per second: 80, episode reward: 5.000, mean reward: 0.500 [0.000, 1.000], mean action: 0.300 [0.000, 1.000], mean observation: 0.300 [0.000, 1.000], loss: --, mean_absolute_error: --, mean_q: --
[0, 0, 0]
[0, 0, 0, 1]


D:\Anaconda3\lib\site-packages\rl\memory.py:29: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


[0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 1]
[0, 0, 0, 1, 1, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1]
  20/500: episode: 2, duration: 0.982s, episode steps: 10, steps per second: 10, episode reward: 7.000, mean reward: 0.700 [0.000, 1.000], mean action: 0.600 [0.000, 1.000], mean observation: 0.550 [0.000, 1.000], loss: 0.269254, mean_absolute_error: 0.327091, mean_q: 0.081300
[0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 1, 1, 0, 1, 1]
[0, 0, 0, 1, 1, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 0, 1, 1, 0, 1]
[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0]
[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
  30/500: episode: 3, duration: 0.096s, episode steps: 10, steps per second: 104, episode reward: 7.000, mean reward: 0.700 [0.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.500 [0.000, 1.000], loss: 0.289700, mean_absolute_error: 0.366818, mean_q: 0.136

D:\Anaconda3\lib\site-packages\rl\memory.py:29: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


[1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 1]
[1, 0, 0, 0, 1, 1, 1]
[1, 0, 0, 0, 1, 1, 1, 0]
[1, 0, 0, 0, 1, 1, 1, 0, 1]
[1, 0, 0, 0, 1, 1, 1, 0, 1, 1]
[1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0]
[1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0]
  40/500: episode: 4, duration: 0.120s, episode steps: 10, steps per second: 83, episode reward: 6.000, mean reward: 0.600 [0.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.500 [0.000, 1.000], loss: 0.241171, mean_absolute_error: 0.362811, mean_q: 0.211130
[1, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]


...

[1, 0, 1, 1, 0, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]
 410/500: episode: 41, duration: 0.111s, episode steps: 10, steps per second: 90, episode reward: 8.000, mean reward: 0.800 [0.000, 1.000], mean action: 0.800 [0.000, 1.000], mean observation: 0.750 [0.000, 1.000], loss: 0.291819, mean_absolute_error: 1.388030, mean_q: 2.919295
[0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 1, 1, 0, 1, 1]
[0, 0, 0, 0, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0]
[0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0]
 420/500: episode: 42, duration: 0.095s, episode steps: 10, steps per second: 105, episode reward: 7.000, mean reward: 0.700 [0.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.500 [0.000, 1.000], loss: 0.190540, mean_absolute_error: 1.362183, mean_q: 2.955077
[1, 0, 1]
[1, 0, 1, 1]
[1, 0,


[1, 0, 1, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
[1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1]
 430/500: episode: 43, duration: 0.111s, episode steps: 10, steps per second: 90, episode reward: 10.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.700 [0.000, 1.000], mean observation: 0.650 [0.000, 1.000], loss: 0.294704, mean_absolute_error: 1.445764, mean_q: 3.144175
[0, 0, 1]
[0, 0, 1, 1]
[0, 0, 1, 1, 0]
[0, 0, 1, 1, 0, 0]
[0, 0, 1, 1, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 1]
[0, 0, 1, 1, 0, 0, 0, 1, 0]
[0, 0, 1, 1, 0, 0, 0, 1, 0, 1]
[0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0]
 440/500: episode: 44, duration: 0.100s, episode steps: 10, steps per second: 100, episode reward: 6.000, mean reward: 0.600 [0.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.500 [0.000, 1.000], loss: 0.261528, mean_absolute_error: 1.424281, mean_q: 3.087670
[1, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0, 1]
[1, 0, 0, 


[1, 0, 0, 0, 1, 1, 0, 1]
[1, 0, 0, 0, 1, 1, 0, 1, 0]
[1, 0, 0, 0, 1, 1, 0, 1, 0, 1]
[1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1]
[1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1]
 450/500: episode: 45, duration: 0.101s, episode steps: 10, steps per second: 99, episode reward: 6.000, mean reward: 0.600 [0.000, 1.000], mean action: 0.600 [0.000, 1.000], mean observation: 0.550 [0.000, 1.000], loss: 0.301147, mean_absolute_error: 1.487703, mean_q: 3.216314
[1, 1, 0]
[1, 1, 0, 1]
[1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 0]
[1, 1, 0, 1, 1, 1, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 1, 0]
[1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0]
 460/500: episode: 46, duration: 0.089s, episode steps: 10, steps per second: 113, episode reward: 8.000, mean reward: 0.800 [0.000, 1.000], mean action: 0.600 [0.000, 1.000], mean observation: 0.650 [0.000, 1.000], loss: 0.249543, mean_absolute_error: 1.530695, mean_q: 3.274679
[1, 0, 1]
[1, 0, 1, 1]
[1, 0, 1, 1, 0]
[1, 0, 1, 1


[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0]
 470/500: episode: 47, duration: 0.091s, episode steps: 10, steps per second: 109, episode reward: 8.000, mean reward: 0.800 [0.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.500 [0.000, 1.000], loss: 0.301767, mean_absolute_error: 1.562073, mean_q: 3.393026
[0, 0, 1]
[0, 0, 1, 1]
[0, 0, 1, 1, 0]
[0, 0, 1, 1, 0, 1]
[0, 0, 1, 1, 0, 1, 1]
[0, 0, 1, 1, 0, 1, 1, 1]
[0, 0, 1, 1, 0, 1, 1, 1, 1]
[0, 0, 1, 1, 0, 1, 1, 1, 1, 0]
[0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1]
[0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0]
 480/500: episode: 48, duration: 0.095s, episode steps: 10, steps per second: 105, episode reward: 6.000, mean reward: 0.600 [0.000, 1.000], mean action: 0.700 [0.000, 1.000], mean observation: 0.700 [0.000, 1.000], loss: 0.248140, mean_absolute_error: 1.583939, mean_q: 3.446417
[1, 0, 1]
[1, 0, 1, 0]
[1, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 1]
[1, 0, 1, 0, 1, 1, 0]
[1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 0, 1, 1, 0, 1, 1]
[1, 0, 1, 0, 1, 1, 0, 1, 1, 0]
[1,

 490/500: episode: 49, duration: 0.086s, episode steps: 10, steps per second: 117, episode reward: 8.000, mean reward: 0.800 [0.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.500 [0.000, 1.000], loss: 0.317350, mean_absolute_error: 1.620282, mean_q: 3.551276
[0, 1, 1]
[0, 1, 1, 0]
[0, 1, 1, 0, 1]
[0, 1, 1, 0, 1, 1]
[0, 1, 1, 0, 1, 1, 0]
[0, 1, 1, 0, 1, 1, 0, 0]
[0, 1, 1, 0, 1, 1, 0, 0, 1]
[0, 1, 1, 0, 1, 1, 0, 0, 1, 1]
[0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]
[0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0]
 500/500: episode: 50, duration: 0.098s, episode steps: 10, steps per second: 103, episode reward: 7.000, mean reward: 0.700 [0.000, 1.000], mean action: 0.600 [0.000, 1.000], mean observation: 0.650 [0.000, 1.000], loss: 0.360748, mean_absolute_error: 1.667379, mean_q: 3.601879
done, took 6.166 seconds


In [29]:
env.reset()
dqn.test(env)
print(env.state_list)

Testing for 1 episodes ...
Episode 1: reward: 10.000, steps: 10
[1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]


In [64]:
env.reset()

array([1, 1])

In [73]:
np.argmax(model.predict(env.state.reshape(1, 1, 2)))

1

In [72]:
env.step(0)

(array([1, 0]), 1, False, {})

In [53]:
model.get_layer(index=0).input

<tf.Tensor 'flatten_2_input:0' shape=(?, 1, 2) dtype=float32>

In [76]:
dqn.test(env, 2, visualize=True)

Testing for 2 episodes ...
[0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Episode 1: reward: 10.000, steps: 10
[1, 1, 0]
[1, 1, 0, 1]
[1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 0]
[1, 1, 0, 1, 1, 0, 1]
[1, 1, 0, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 0, 1, 1, 0]
[1, 1, 0, 1, 1, 0, 1, 1, 0, 1]
[1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
[1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
Episode 2: reward: 10.000, steps: 10
